# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"
df = pd.read_csv(output_data_file)

del df['Unnamed: 0']
df

,city,latitude,longitude,max_temperature,humidity,cloudiness,wind_speed
0,Kapaa,22.08,-159.32,23.00,78,75,2.60
1,Osypenko,46.91,36.83,7.78,100,0,3.13
2,Imbituba,-28.24,-48.67,17.00,63,1,1.00
3,Busselton,-33.65,115.33,25.56,36,39,5.58
4,Bac Lieu,9.29,105.72,31.12,58,37,5.48
...,...,...,...,...,...,...,...
579,Ofakim,31.31,34.62,19.44,68,0,0.45
580,Saýat,38.78,63.88,17.00,72,90,3.10
581,Broken Hill,-31.95,141.43,18.00,27,52,3.60
582,Abnūb,27.27,31.15,24.30,24,0,5.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
#Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]

# fill Nan values and convert to float
humidities = df["humidity"].astype(float)

In [33]:

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
# Narrow down the DataFrame to find your ideal weather condition. 
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

df_new = df.loc[(df['humidity'] <= 10) & (df['wind_speed'] <= 4) & (df['cloudiness'] <= 20)]

df_new = df_new.reset_index(drop=True)
df_new

,city,latitude,longitude,max_temperature,humidity,cloudiness,wind_speed
0,Yumen,40.28,97.20,18.38,10,17,2.54
1,Gariāband,20.64,82.06,40.83,7,0,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
#Store into variable named hotel_df. Add a "Hotel Name" column to the DataFrame.
df_new ["hotel name"] = ""
df_new ["hotel address"] = ""
df_new.head()

,city,latitude,longitude,max_temperature,humidity,cloudiness,wind_speed,hotel name,hotel address
0,Yumen,40.28,97.20,18.38,10,17,2.54,,
1,Gariāband,20.64,82.06,40.83,7,0,4.00,,


In [83]:
import json
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

In [93]:
# Use the lat/lng we recovered to identify hotels
for index, row in df_new.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # get lat, lng from df_new
    city = row["city"]
    lat = row["latitude"]
    lng = row["longitude"]
      
    # update address key value 
    params["location"] = f"{lat},{lng}"

    # make request 
    name_address = requests.get(base_url, params=params)
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f"Processing Record {index} | {city}")
        df_new.loc[index, "city"] = city
        df_new.loc[index, "hotel name"] = name_address["results"][0]["hotel"]
        df_new.loc[index, "hotel address"] = name_address["results"][0]["place"]      
      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Processing Record 0 | Yumen
Missing field/result... skipping.
Processing Record 1 | Gariāband
Missing field/result... skipping.


In [96]:
# Save Data to csv
df_new.to_csv("Hotel_Output.csv")

# Visualize to confirm airport data appears
df_new.head()

,city,latitude,longitude,max_temperature,humidity,cloudiness,wind_speed,hotel name,hotel address
0,Yumen,39.73287,98.494548,18.38,10,17,2.54,Jiuquan,Jiuquan
1,Gariāband,20.63479,82.061497,40.83,7,0,4.00,Gariyaband,Gariyaband


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_new.iterrows()]

In [101]:
# Add marker layer ontop of heat map
marker_locations = df_new[["latitude", "longitude"]]
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)


fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))